In [1]:
import re
import time
import string

## Textual algorithms - lab01

### used algorithms

#### naive algorithm


In [2]:
def find_pattern_naive(text:string, pattern:string):
    # lets find all corrects shifts
    result = []

    for s in range(0, len(text) - len(pattern) + 1):
        if pattern == text[s:s+len(pattern)]:
            result.append(s)
    
    return result

#### kmp


In [3]:
def prefix_function(pattern:string):
    lps = [0] * len(pattern)
    l = 0
    i = 1

    while i < len(pattern):
        while l > 0 and pattern[i] != pattern[l]:
            l -= 1
        
        if pattern[i] == pattern[l]:
            l += 1
            
        lps[i] = l
        i += 1
    
    return lps

def find_pattern_kmp(text:string, pattern:string):
    lps = prefix_function(pattern)
    result = []
    i = 0
    j = 0

    while i < len(text):
        if text[i] != pattern[j]:
            if j > 0:
                j = lps[j-1]
            else:
                i += 1
        else:
            i, j = i+1, j+1
            if j == len(pattern):
                result.append(i-j)
                j = lps[j-1]
    
    return result

#### finite automaton

In [4]:
def transition_table(pattern:string):
    result = []
    alphabet = set()
    
    for a in pattern:
        alphabet.add(a)
    
    for q in range(len(pattern) + 1):
        result.append({})
        for a in alphabet:
            k = min(len(pattern), q+1)
            suffix = pattern[:q] + a
            while pattern[:k] != "" and pattern[:k] != suffix[q - k + 1:]:
                k -= 1
            result[q][a] = k
    return result

def find_pattern_finite_automaton(text:string, pattern:string):
    q = 0
    delta = transition_table(pattern)
    result = []
    for s in range(0, len(text)):
        if text[s] in delta[q]:
            q = delta[q][text[s]]
            if q == len(delta) - 1:
                result.append(s + 1 - q)
        else:
            q = 0
    
    return result

### find word "art" in given file

In [5]:
with open('1997_714.txt', 'r', encoding='utf8') as file:
    data = file.read()
    search_pattern = 'art'
    
    result_naive      = find_pattern_naive(data, search_pattern)
    result_automaton  = find_pattern_finite_automaton(data, search_pattern)
    result_kmp        = find_pattern_kmp(data, search_pattern)
    
    if result_naive == result_automaton and result_naive == result_kmp:
        print('Each of the algorithms gave the same result! (good one hopefully)')
        print(result_naive)
    else:
        print('Differents in results!')

Each of the algorithms gave the same result! (good one hopefully)
[1156, 1505, 4692, 4734, 4879, 5082, 5148, 5949, 6039, 7266, 7511, 7781, 8044, 8299, 9104, 9959, 10022, 10224, 11122, 11207, 11618, 13194, 15284, 15358, 16092, 16261, 16406, 16547, 16616, 16840, 16856, 23637, 24061, 24152, 24586, 24683, 24780, 24931, 25530, 25689, 27001, 27288, 27479, 27542, 27592, 27857, 28373, 28558, 28766, 30964, 31021, 31096, 31362, 31811, 32609, 32968, 33053, 33268, 33595, 34651, 34737, 35511, 36155, 37143, 37543, 38451, 38595, 39056, 39210, 39436, 39568, 39980, 41152, 41829, 42028, 42198, 42371, 42504, 42718, 42896, 42941, 43447, 43555, 43787, 44590, 44653, 44953, 45010, 45293, 45401, 47319, 47422, 48785, 48820, 48906, 49052, 49259, 49316, 49488, 49559, 49915, 49979, 50102, 50160, 50702, 51050, 51179, 51966, 52071, 52272, 52552, 53008, 53032, 53211, 53788, 53931, 54078, 54137, 54770, 55075, 55279, 55465, 55807, 55991, 56827, 56911, 57164, 57549, 57800, 57932, 57989, 58280, 58378, 58874, 58966, 5939

### compare execution time of algorithms #1

In [6]:
with open('1997_714.txt', 'r', encoding='utf8') as file:
    data = file.read()
    search_pattern = 'art'
    
    time_start = time.time()
    find_pattern_naive(data, search_pattern)
    time_end   = time.time()
    print(f"execution time of naive algorithm: {time_end - time_start}")
    
    time_start = time.time()
    find_pattern_finite_automaton(data, search_pattern)
    time_end   = time.time()
    print(f"execution time of finite automaton algorithm: {time_end - time_start}")
    
    time_start = time.time()
    find_pattern_kmp(data, search_pattern)
    time_end   = time.time()
    print(f"execution time of kmp algorithm: {time_end - time_start}")

execution time of naive algorithm: 0.03700828552246094
execution time of finite automaton algorithm: 0.01700425148010254
execution time of kmp algorithm: 0.03500795364379883


### compare execution time of algorithms #2

In [7]:
with open('passages-head.tsv', 'r', encoding='utf8') as file:
    data = file.read()
    search_pattern = 'Ukraina'
    
    time_start = time.time()
    find_pattern_naive(data, search_pattern)
    time_end   = time.time()
    print(f"execution time of naive algorithm: {time_end - time_start}")
    
    time_start = time.time()
    find_pattern_finite_automaton(data, search_pattern)
    time_end   = time.time()
    print(f"execution time of finite automaton algorithm: {time_end - time_start}")
    
    time_start = time.time()
    find_pattern_kmp(data, search_pattern)
    time_end   = time.time()
    print(f"execution time of kmp algorithm: {time_end - time_start}")

execution time of naive algorithm: 11.824442625045776
execution time of finite automaton algorithm: 7.494645357131958
execution time of kmp algorithm: 11.184900522232056


### kmp 5 times faster that automaton (preprocessing)

In [8]:
pattern = "Kraków to stolica Polski Perła ukryta we mgle Z najdalszych krain zamorskich Każdy wspomnienia tu śle"


time_start = time.time()
transition_table(pattern)
time_end   = time.time()
print(f"finite automaton preprocessing time: {time_end - time_start}")

time_start = time.time()
prefix_function(pattern)
time_end   = time.time()
print(f"kmp preprocessing time: {time_end - time_start}")


finite automaton preprocessing time: 0.04200935363769531
kmp preprocessing time: 0.0


### naive 5 times slower than other algorithms

In [17]:
def find_pattern_finite_automaton_time_measure(text:string, pattern:string):
    q = 0
    delta = transition_table(pattern)
    time_start = time.time()
    result = []
    for s in range(0, len(text)):
        if text[s] in delta[q]:
            q = delta[q][text[s]]
            if q == len(delta) - 1:
                result.append(s + 1 - q)
        else:
            q = 0
    time_end  = time.time()
    print(f"finite automaton time: {time_end - time_start}")
    

def find_pattern_kmp_time_measure(text:string, pattern:string):
    lps = prefix_function(pattern)
    result = []
    i = 0
    j = 0
    time_start = time.time()
    while i < len(text):
        if text[i] != pattern[j]:
            if j > 0:
                j = lps[j-1]
            else:
                i += 1
        else:
            i, j = i+1, j+1
            if j == len(pattern):
                result.append(i-j)
                j = lps[j-1]
    time_end  = time.time()
    print(f"kmp automaton time: {time_end - time_start}")

    
pattern = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"*1000
text = "babababababababababa"*100000

find_pattern_finite_automaton_time_measure(text, pattern)
find_pattern_kmp_time_measure(text, pattern)
start_time = time.time()
find_pattern_naive(text, pattern)
end_time = time.time()
print(f"naive algorithm time: {end_time - start_time}")

finite automaton time: 0.22605180740356445
kmp automaton time: 0.46210432052612305
naive algorithm time: 2.853322744369507
